In [18]:
import numpy as np 
import pandas as pd 
import torch 
import torch.nn as nn
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

In [13]:
#load data
(X_train,Y_train),(X_test,Y_test) = mnist.load_data()
#X 为图片 ，2维  y  为 label 
print(X_train.shape,Y_train.shape,Y_train[0])


(60000, 28, 28) (60000,) 5


In [15]:
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import time
import torch.nn.functional as F
# training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), # 隨機將圖片水平翻轉
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
])
# testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [23]:
batch_size=64
train_set = ImgDataset(X_train,Y_train,train_transform)
val_set = ImgDataset(X_test,Y_test,test_transform)
train_loader = DataLoader(train_set,batch_size = batch_size,shuffle=True)
val_loader = DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [30]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [64,1, 28, 28]
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, 3, 1, 1),  # [64, 28, 28]
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 14, 14]

            nn.Conv2d(16, 32, 3, 1, 1), # [128, 14, 14]
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 7, 7]
            
            nn.Conv2d(32, 64, 3, 1, 1), # [512, 7, 7]
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.fc = nn.Sequential(
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [31]:
#setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
# print(device)
model = Classifier().to(device)
cirection = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)


In [33]:
epochs = 30
for epoch in range(epochs):
    epoch_start_time = time.time()
    train_acc =0.0
    val_acc =0.0
    train_loss = 0.0
    val_loss =0.0
    model.train()
    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        x,y = data[0].to(device),data[1].to(device)
        #print(x.shape)
        y_pred = model(x)
        loss = cirection(y_pred,y.long())
        loss.backward()
        optimizer.step()
        train_acc +=np.sum(np.argmax(y_pred.cpu().data.numpy(),axis=1)== y.cpu().numpy())
        train_loss +=loss.item()
    model.eval()
    with torch.no_grad():
        for i,data in enumerate(val_loader):
            valx ,valy = data[0].to(device),data[1].to(device)
            val_pred = model(valx)
            batch_loss = cirection(val_pred,valy.long())
            val_acc +=np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1)== valy.cpu().numpy())
            val_loss +=batch_loss.item()

        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, epochs, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))
    model.train()

[001/030] 15.55 sec(s) Train Acc: 0.962183 Loss: 0.001858 | Val Acc: 0.969400 loss: 0.001423
[002/030] 15.18 sec(s) Train Acc: 0.973250 Loss: 0.001290 | Val Acc: 0.983600 loss: 0.000740
[003/030] 17.97 sec(s) Train Acc: 0.978767 Loss: 0.001059 | Val Acc: 0.988100 loss: 0.000551
[004/030] 18.44 sec(s) Train Acc: 0.980717 Loss: 0.000947 | Val Acc: 0.987800 loss: 0.000591
[005/030] 18.27 sec(s) Train Acc: 0.983133 Loss: 0.000816 | Val Acc: 0.982800 loss: 0.000839
[006/030] 16.99 sec(s) Train Acc: 0.984167 Loss: 0.000749 | Val Acc: 0.983900 loss: 0.000789
[007/030] 15.35 sec(s) Train Acc: 0.986200 Loss: 0.000671 | Val Acc: 0.988600 loss: 0.000580
[008/030] 15.34 sec(s) Train Acc: 0.986583 Loss: 0.000635 | Val Acc: 0.987600 loss: 0.000583
[009/030] 15.50 sec(s) Train Acc: 0.987433 Loss: 0.000602 | Val Acc: 0.988200 loss: 0.000612
[010/030] 15.88 sec(s) Train Acc: 0.988150 Loss: 0.000587 | Val Acc: 0.984600 loss: 0.000842
[011/030] 14.99 sec(s) Train Acc: 0.988917 Loss: 0.000537 | Val Acc: 0